# Kompetisi :

Dalam latihan ini, peserta perlu menyetel dan berupaya meningkatkan akurasi keseluruhan model.

Untuk membantu Anda memulai, berikut beberapa parameter yang berfungsi untuk membuat model lebih efisien.

- Epochs  
- Batch Size 
- Optimizers : Kita telah menggunakan SGD sebagai pengoptimal. Peserta dapat mencoba menerapkan pengoptimal lain dan menguji untuk mendapatkan konvergensi cepat.
- Data Augmentation : Ingat, bahwa kita memiliki kumpulan data yang tidak seimbang. Peserta dapat mencoba teknik augmentasi yang berbeda untuk kelas minoritas.
- Model : Jika Anda telah mengeksploitasi semua metode di atas untuk meningkatkan model Anda, Anda dapat mengubah model dengan menambahkan lebih banyak Lapisan ke dalamnya dan lihat apakah hal tersebut meningkatkan akurasi tersebut.

Perhatikan, sebelum Anda mulai mengutak-atik dan melatih model Anda, ada baiknya merujuk ke <i>notebook</i> berikut terlebih dahulu untuk melihat bagaimana pengaruhnya terhadap model Anda:

[Epochs impact on Overfitting](https://datascience.stackexchange.com/questions/27561/can-the-number-of-epochs-influence-overfitting ) 


[Effect of Batch Size on Training Dynamics](https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e)

[Introduction to Optimizers](https://algorithmia.com/blog/introduction-to-optimizers)

# Melatih Model dengan Data Augmentasi: 


Kita membuat fungsi baru bernama `augmentation(name,category,filenames,labels,i)` dan di sini peserta dapat menambahkan lebih banyak sampel ke Kategori yang memiliki data tidak seimbang. 

In [ ]:
import sys
sys.path.append('/workspace/python/source_code')

from utils import * 
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def augmentation(name,category,filenames,labels,i):
    # Konstanta Penting
    file_path = "Dataset/Aug/"
    images = []
    (h, w) = (232,232)
    center = (w / 2, h / 2)
    angle90 = 90
    angle180 = 180
    angle270 = 270
    scale = 1.0
    img = load_image(name , interpolation = cv2.INTER_LINEAR)
    
    ## ~~ Tambahkan Augmentasi disini ~~
    if category == 0 :
        images.append(cv2.flip(img,0))
    elif category == 1 :
        pass
    elif category == 2 :
        pass
    elif category == 3 :
        pass
    elif category == 4 :
        pass
    elif category == 5 :
        pass
    elif category == 6 :
        pass
    elif category == 7 :
        images.append(cv2.flip(img,0))
        
    ## ~~ Akhir kode Augmentasi  ~~
    for j in range(len(images)):
        cv2.imwrite(file_path+str(i+j)+'.jpeg',images[j])
        filenames.append(file_path+str(i+j)+'.jpeg')
        labels.append(category)
    i = i + len(images)
    return i



##### Fungsi ini akan diteruskan ke fungsi `load_dataset()` untuk menghasilkan augmentasi ini.

Silakan tunggu beberapa menit sementara diperlukan untuk mengaugmentasi gambar.

In [ ]:
filenames,labels = load_dataset(augment_fn = augmentation)

In [ ]:
# Setel Ukuran set Validasi
val_filenames , val_labels = make_test_set(filenames,labels,val=0.1)

In [ ]:
#Buat set Uji dan Latih
test = 0.1
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(filenames, labels, test_size=test, random_state=1)

In [ ]:
import tensorflow as tf
y_train = tf.one_hot(y_train,depth=8)
y_test = tf.one_hot(y_test,depth=8)
val_labels = tf.one_hot(val_labels,depth=8)

In [ ]:
# Jadikan Dataset kompatibel dengan Tensorflow Data Pipelining.

# ~~ Ganti ukuran batch Size disini ~~
batch_size = 64
# ~~ Ganti ukuran batch Size disini ~~

train,test,val = make_dataset((x_train,y_train,batch_size),(x_test,y_test,32),(val_filenames,val_labels,32))

# Arsitektur Model :



In [ ]:
import numpy as np
tf.random.set_seed(1337)

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten ,Dropout, MaxPooling2D
from tensorflow.keras import backend as K 

#Reset Graphs dan buat Sequential model

K.clear_session()
model = Sequential()

## ~~ Ganti arsitektur model atau Parameter disini
#Convolution Layers

model.add(Conv2D(64, kernel_size=10,strides=3, activation='relu', input_shape=(232,232,3)))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(256, kernel_size=5,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Conv2D(288, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))
model.add(Conv2D(272, kernel_size=3,strides=1,padding='same',activation='relu'))
model.add(Conv2D(256, kernel_size=3,strides=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))
model.add(Dropout(0.5))
model.add(Flatten())

#Linear Layers 

model.add(Dense(3584,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(8, activation='softmax'))


## ~~ Ganti arsitektur model atau Parameter disini

# Print Model Summary

model.summary()

In [ ]:
import functools

#Definisikan jumlah Epochs

## ~~ Ganti jumlah Epochs disini ~~
epochs = 24
## ~~ Ganti jumlah Epochs disini ~~


# Include Top-2 Accuracy Metrics 
top2_acc = functools.partial(tensorflow.keras.metrics.top_k_categorical_accuracy, k=2)
top2_acc.__name__ = 'top2_acc'


## ~~ Ganti Optimizer atau Parameters disini
# Optimizer
sgd = tensorflow.keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)
## ~~ Ganti Optimizer atau Parameters disini

#Compile Model dengan Loss Function , Optimizer dan Metrics
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, 
              optimizer=sgd,
              metrics=['accuracy',top2_acc])

# Latih Model
trained_model = model.fit(train,
          epochs=epochs,
          verbose=1,
          validation_data=val)

# Pengetesan model dengan data Validasi
score = model.evaluate(test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Visualisasi

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(15,5))
ax = f.add_subplot(121)
ax.plot(trained_model.history['accuracy'])
ax.plot(trained_model.history['val_accuracy'])
ax.set_title('Model Accuracy')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.legend(['Train', 'Val'])

ax2 = f.add_subplot(122)
ax2.plot(trained_model.history['loss'])
ax2.plot(trained_model.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Val'],loc= 'upper left')

plt.show()

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
import pandas as pd

#Visualisasi heatmap dengan confusion metrik
pred = model.predict(val)
p = np.argmax(pred, axis=1)
y_valid = np.argmax(val_labels, axis=1, out=None)
results = confusion_matrix(y_valid, p) 
classes=['NC','TD','TC','H1','H3','H3','H4','H5']
df_cm = pd.DataFrame(results, index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (15,15))

sn.heatmap(df_cm, annot=True, cmap="Blues")

Let us now save our Model and the trained Weights for Future usage :

In [ ]:
#Menyimpan model
model.save('cyc_pred_comp.h5')

### Bootcamp lain
Konten Bootcamp ini berasal dari [OpenACC GPU Bootcamp Github](https://github.com/gpuhackathons-org/gpubootcamp). Berikut beberapa Bootcamp tambahan yang mungkin menarik:

- [Physics Informed Neural Network](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/hpc_ai/PINN)

## License
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).
